In [6]:
# -*- coding: utf-8 -*-
"""
Este script genera un dashboard interactivo de un modelo económico (DD-AA y Mercado Cambiario)
utilizando la librería Bokeh. El resultado es un único archivo HTML interactivo.
"""

# =============================================================================
# --- PASO 1: IMPORTACIÓN DE LIBRERÍAS ---
# =============================================================================
# Se importan las herramientas necesarias de Bokeh para crear la aplicación y de NumPy para cálculos numéricos.

from bokeh.plotting import figure, show # Función principal para crear gráficos y mostrar el dashboard.
from bokeh.io import output_notebook # Función para configurar la salida en el notebook.
from bokeh.layouts import layout, column, row # Funciones para organizar los componentes del dashboard.
from bokeh.models import (ColumnDataSource, CustomJS, Slider, Select,  # Modelos para datos, JS personalizado y widgets.
                            HoverTool, Div, Arrow, VeeHead, Label)  # Modelos para interactividad, anotaciones y etiquetas.
import numpy as np # Librería para operaciones matemáticas y manejo de arrays.

# --- Configurar Bokeh para mostrar en el cuaderno ---
# Esta línea es crucial para que show() muestre el gráfico en la celda y no en una nueva pestaña.
output_notebook()

# =============================================================================
# --- PASO 2: DEFINICIÓN DE PARÁMETROS Y CÁLCULO INICIAL ---
# =============================================================================
# Se definen las constantes del modelo económico y se calcula el punto de equilibrio inicial.

# --- Parámetros Fijos del Modelo ---
RANGO_Y = np.linspace(0, 25, 100) # Rango de valores para el eje de Producción (Y).
RANGO_I = np.linspace(0, 15, 100) # Rango de valores para el eje de Tasa de Interés (i).
IP_INTERCEPT = 20                # Intercepto de la curva de Paridad de Intereses (IP).
IP_PENDIENTE = 1.5               # Pendiente de la curva IP.
AA_INTERCEPT = 22                # Intercepto de la curva AA (mercado de activos).
AA_PENDIENTE = 1.0               # Pendiente de la curva AA.
DD_INTERCEPT = 2                 # Intercepto de la curva DD (mercado de bienes).
DD_PENDIENTE = 0.8               # Pendiente de la curva DD.

# --- Cálculo del Equilibrio Inicial (Eq-1) ---
# Se resuelve el sistema de ecuaciones para encontrar los valores iniciales de Y, e, i.
y_eq1 = (AA_INTERCEPT - DD_INTERCEPT) / (DD_PENDIENTE + AA_PENDIENTE)
e_eq1 = DD_INTERCEPT + DD_PENDIENTE * y_eq1
i_eq1 = (IP_INTERCEPT - e_eq1) / IP_PENDIENTE


# =============================================================================
# --- PASO 3: CREACIÓN DE FUENTES DE DATOS (ColumnDataSource) ---
# =============================================================================
# ColumnDataSource es el objeto fundamental de Bokeh para la interactividad.
# Contiene los datos que los gráficos mostrarán y que el código JavaScript podrá modificar.

# --- Fuentes para las Curvas del Modelo ---
# Curva IP (Mercado Cambiario)
source_ip = ColumnDataSource(data=dict(x=RANGO_I, y=IP_INTERCEPT - IP_PENDIENTE * RANGO_I))
source_ip_nueva = ColumnDataSource(data=dict(x=[], y=[])) # Vacía, se llenará con el shock.
# Curva DD (Mercado de Bienes)
source_dd = ColumnDataSource(data=dict(x=RANGO_Y, y=DD_INTERCEPT + DD_PENDIENTE * RANGO_Y))
source_dd_nueva = ColumnDataSource(data=dict(x=[], y=[])) # Vacía, se llenará con el shock.
# Curva AA (Mercado de Activos)
source_aa = ColumnDataSource(data=dict(x=RANGO_Y, y=AA_INTERCEPT - AA_PENDIENTE * RANGO_Y))
source_aa_nueva = ColumnDataSource(data=dict(x=[], y=[])) # Vacía, se llenará con el shock.

# --- Fuentes para los Puntos de Equilibrio ---
# Punto de equilibrio inicial (Eq-1).
source_eq1 = ColumnDataSource(data=dict(y=[y_eq1], e=[e_eq1], i=[i_eq1]))
# Punto de equilibrio final (Eq-2), inicialmente vacío.
source_eq2 = ColumnDataSource(data=dict(y=[], e=[], i=[]))

# --- Fuentes para Anotaciones Gráficas ---
# Flecha que muestra el ajuste del Eq-1 al Eq-2 en el mercado cambiario.
source_flecha_mc = ColumnDataSource(data=dict(x_start=[], y_start=[], x_end=[], y_end=[]))
# Flecha que muestra el ajuste del Eq-1 al Eq-2 en el modelo DD-AA.
source_flecha_ddaa = ColumnDataSource(data=dict(x_start=[], y_start=[], x_end=[], y_end=[]))


# Líneas de proyección del equilibrio inicial (Eq-1) a los ejes.
source_eq1_lines_mc = ColumnDataSource(data=dict(x0=[0, i_eq1], y0=[e_eq1, 0], x1=[i_eq1, i_eq1], y1=[e_eq1, e_eq1]))
source_eq1_lines_ddaa = ColumnDataSource(data=dict(x0=[0, y_eq1], y0=[e_eq1, 0], x1=[y_eq1, y_eq1], y1=[e_eq1, e_eq1]))

# Líneas de proyección del equilibrio final (Eq-2), inicialmente vacías.
source_eq2_lines_mc = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[]))
source_eq2_lines_ddaa = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[]))


# =============================================================================
# --- PASO 4: CREACIÓN DE LAS FIGURAS (GRÁFICOS) ---
# =============================================================================
# Se definen los dos gráficos principales y se les añaden los elementos visuales (glifos).

# --- Herramienta Hover (Información al pasar el ratón) ---
# Define qué información se mostrará al pasar el cursor sobre los puntos de equilibrio.
hover_tool = HoverTool(tooltips=[("Producción (Y)", "@y{0.2f}"),
                                   ("Tipo de Cambio (e)", "@e{0.2f}"),
                                   ("Tasa de Interés (i)", "@i{0.2f}")])

# --- Gráfico 1: Mercado Cambiario ---
p_mc = figure(height=400, width=500, title="Mercado Cambiario",
                x_range=(0, 15), y_range=(-5, 30),
                x_axis_label="Tasa de Interés (i)", y_axis_label="Tipo de Cambio (e)",
                tools=[hover_tool, "pan,wheel_zoom,box_zoom,reset,save"])

# Dibuja las líneas, segmentos, puntos y flecha, vinculando cada uno a su fuente de datos.
p_mc.line('x', 'y', source=source_ip, line_width=2, color='darkblue', legend_label='IP (Retorno Depósitos Nacionales)')
p_mc.line('x', 'y', source=source_ip_nueva, line_width=2, color='darkblue', line_dash='dashed', legend_label="IP' (Nueva)")
p_mc.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq1_lines_mc, line_width=1, color='gray', line_dash='dashed')
p_mc.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq2_lines_mc, line_width=1, color='lightcoral', line_dash='dashed')
p_mc.scatter(x='i', y='e', source=source_eq1, size=10, color='black', legend_label='Equilibrio Inicial (Eq-1)', marker='circle')
p_mc.scatter(x='i', y='e', source=source_eq2, size=10, color='red', legend_label='Equilibrio Final (Eq-2)', marker='circle')
flecha_mc_renderer = Arrow(end=VeeHead(size=15, fill_color="darkmagenta"),
                             x_start='x_start', y_start='y_start', x_end='x_end', y_end='y_end',
                             source=source_flecha_mc, line_color="darkmagenta", line_width=2)
p_mc.add_layout(flecha_mc_renderer)

# Añade una etiqueta con la fórmula de la curva IP usando LaTeX.
ip_label = Label(x=14.5, y=-4, text_align='right',
                 text='$$e = I_{IP} - P_{IP} \\cdot i$$',
                 text_font_size='8pt', text_color='darkblue',
                 background_fill_color='white', background_fill_alpha=0.7)
p_mc.add_layout(ip_label)

# --- Gráfico 2: Modelo DD-AA ---
p_ddaa = figure(height=400, width=500, title="Modelo DD-AA",
                  x_range=(0, 25), y_range=(-5, 30),
                  x_axis_label="Producción (Y)", y_axis_label="Tipo de Cambio (e)",
                  tools=[hover_tool, "pan,wheel_zoom,box_zoom,reset,save"])

# Dibuja las líneas, segmentos y puntos, vinculando cada uno a su fuente de datos.
p_ddaa.line('x', 'y', source=source_dd, line_width=2, color='darkgreen', legend_label='Curva DD')
p_ddaa.line('x', 'y', source=source_dd_nueva, line_width=2, color='darkgreen', line_dash='dashed', legend_label="DD' (Nueva)")
p_ddaa.line('x', 'y', source=source_aa, line_width=2, color='crimson', legend_label='Curva AA')
p_ddaa.line('x', 'y', source=source_aa_nueva, line_width=2, color='crimson', line_dash='dashed', legend_label="AA' (Nueva)")
p_ddaa.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq1_lines_ddaa, line_width=1, color='gray', line_dash='dashed')
p_ddaa.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq2_lines_ddaa, line_width=1, color='lightcoral', line_dash='dashed')
p_ddaa.scatter(x='y', y='e', source=source_eq1, size=10, color='black', legend_label='Equilibrio Inicial (Eq-1)', marker='circle')
p_ddaa.scatter(x='y', y='e', source=source_eq2, size=10, color='red', legend_label='Equilibrio Final (Eq-2)', marker='circle')
flecha_ddaa_renderer = Arrow(end=VeeHead(size=15, fill_color="darkmagenta"),
                             x_start='x_start', y_start='y_start', x_end='x_end', y_end='y_end',
                             source=source_flecha_ddaa, line_color="darkmagenta", line_width=2)
p_ddaa.add_layout(flecha_ddaa_renderer)

# Añade etiquetas con las fórmulas de las curvas DD y AA.
dd_label = Label(x=24.5, y=-2, text_align='right',
                 text='$$e = I_{DD} + P_{DD} \\cdot Y$$',
                 text_font_size='8pt', text_color='darkgreen',
                 background_fill_color='white', background_fill_alpha=0.7)

aa_label = Label(x=24.5, y=-4, text_align='right',
                 text='$$e = I_{AA} - P_{AA} \\cdot Y$$',
                 text_font_size='8pt', text_color='crimson',
                 background_fill_color='white', background_fill_alpha=0.7)

p_ddaa.add_layout(dd_label)
p_ddaa.add_layout(aa_label)


# --- Ajustes Estéticos Comunes para Ambos Gráficos ---
for p in [p_mc, p_ddaa]:
      p.legend.location = "top_right" # Posición de la leyenda.
      p.legend.click_policy = "hide" # Permite ocultar elementos al hacer clic en la leyenda.
      p.title.align = 'center' # Alineación del título.
      p.title.text_font_size = '14pt' # Tamaño de fuente del título.


# =============================================================================
# --- PASO 5: CREACIÓN DE WIDGETS INTERACTIVOS ---
# =============================================================================
# Se crean los controles que el usuario manipulará para interactuar con el dashboard.

# Widget de menú desplegable para seleccionar el tipo de shock.
shock_select = Select(title="Tipo de Shock:", value="Ninguno", options=['Ninguno', 'Fiscal', 'Monetario', 'Cambiario'])

# Widget de deslizador para ajustar la magnitud del shock.
shock_slider = Slider(start=-5, end=5, value=0, step=0.5, title="Magnitud del Shock")

# Widget Div para mostrar texto HTML (las explicaciones del modelo).
explanation_div = Div(text="""
      <h3>Bienvenido al Dashboard del Modelo DD-AA</h3>
      <p>
      Selecciona un <b>tipo de shock</b> y ajusta su <b>magnitud</b> usando los controles de arriba
      para observar cómo se ajusta la economía a un nuevo equilibrio. Las explicaciones del mecanismo
      de ajuste aparecerán aquí.
      </p>
      """, width=1000, height=200)


# =============================================================================
# --- PASO 6: LÓGICA DE INTERACCIÓN CON CustomJS ---
# =============================================================================
# Este es el cerebro del dashboard. El código JavaScript se ejecuta en el navegador del usuario
# cada vez que se cambia el valor de un widget, actualizando los datos en los ColumnDataSource.

callback = CustomJS(args=dict(
      # Se pasan todos los objetos de Python (fuentes de datos, widgets) al entorno de JavaScript.
      source_ip_n=source_ip_nueva,
      source_dd_n=source_dd_nueva,
      source_aa_n=source_aa_nueva,
      source_eq1=source_eq1,
      source_eq2=source_eq2,
      source_flecha_mc=source_flecha_mc,
      source_flecha_ddaa=source_flecha_ddaa,
      source_eq2_lines_mc=source_eq2_lines_mc,
      source_eq2_lines_ddaa=source_eq2_lines_ddaa,
      shock_select=shock_select,
      shock_slider=shock_slider,
      explanation_div=explanation_div,
      RANGO_I=RANGO_I,
      RANGO_Y=RANGO_Y,
      params={
          'IP_I': IP_INTERCEPT, 'IP_P': IP_PENDIENTE,
          'AA_I': AA_INTERCEPT, 'AA_P': AA_PENDIENTE,
          'DD_I': DD_INTERCEPT, 'DD_P': DD_PENDIENTE
      }
  ), code="""
      // --- Obtener valores actuales de los widgets y parámetros ---
      const shock_type = shock_select.value;
      const shock_value = shock_slider.value;
      const { IP_I, IP_P, AA_I, AA_P, DD_I, DD_P } = params; // Desestructuración para fácil acceso.
      const y_eq1 = source_eq1.data.y[0];
      const e_eq1 = source_eq1.data.e[0];
      const i_eq1 = source_eq1.data.i[0];

      // --- Reiniciar todos los datos dinámicos ---
      // Cada vez que se activa el callback, se limpian los elementos del shock anterior.
      source_ip_n.data = { x: [], y: [] };
      source_dd_n.data = { x: [], y: [] };
      source_aa_n.data = { x: [], y: [] };
      source_eq2.data = { y: [], e: [], i: [] };
      source_flecha_mc.data = { x_start: [], y_start: [], x_end: [], y_end: [] };
      source_flecha_ddaa.data = { x_start: [], y_start: [], x_end: [], y_end: [] };
      source_eq2_lines_mc.data = { x0: [], y0: [], x1: [], y1: [] };
      source_eq2_lines_ddaa.data = { x0: [], y0: [], x1: [], y1: [] };

      // --- Variables para el nuevo equilibrio y textos ---
      let y_eq2, e_eq2, i_eq2;
      let titulo = "<h3>Análisis de Equilibrio</h3>";
      let cuerpo = "<p>Selecciona un tipo de shock para ver el mecanismo de ajuste.</p>";

      // --- Lógica Principal: Calcular nuevo equilibrio según el shock ---
      if (shock_type === 'Fiscal') {
          const nuevo_dd_i = DD_I + shock_value;
          y_eq2 = (AA_I - nuevo_dd_i) / (DD_P + AA_P);
          e_eq2 = nuevo_dd_i + DD_P * y_eq2;
          i_eq2 = (IP_I - e_eq2) / IP_P;

          // Actualiza la fuente de datos de la nueva curva DD y las flechas de ajuste.
          source_dd_n.data = { x: RANGO_Y, y: RANGO_Y.map(y => nuevo_dd_i + DD_P * y) };
          source_flecha_mc.data = { x_start: [i_eq1], y_start: [e_eq1], x_end: [i_eq2], y_end: [e_eq2] };
          source_flecha_ddaa.data = { x_start: [y_eq1], y_start: [e_eq1], x_end: [y_eq2], y_end: [e_eq2] };

          // Define los textos explicativos.
          if (shock_value > 0) {
              titulo = "<h3>Mecanismo de Ajuste: Política Fiscal Expansiva</h3>";
              cuerpo = `1. <b>Inicio</b>: Aumenta el gasto autónomo (G ↑).<br>2. <b>Mercado de Bienes</b>: La Demanda Agregada (DA) aumenta, desplazando la curva DD a la derecha.<br>3. <b>Mercado de Dinero</b>: El aumento en la producción (Y ↑) incrementa la demanda de dinero, elevando la tasa de interés (i ↑).<br>4. <b>Mercado Cambiario</b>: La mayor tasa de interés atrae capitales, apreciando el tipo de cambio (e ↓).<br>5. <b>Resultado Final</b>: Nuevo equilibrio con mayor producción (Y₁ > Y₀) y un tipo de cambio más apreciado (e₁ < e₀).`;
          } else if (shock_value < 0) {
              titulo = "<h3>Mecanismo de Ajuste: Política Fiscal Contractiva</h3>";
              cuerpo = `1. <b>Inicio</b>: Disminuye el gasto autónomo (G ↓).<br>2. <b>Mercado de Bienes</b>: La Demanda Agregada (DA) disminuye, desplazando la curva DD a la izquierda.<br>3. <b>Mercado de Dinero</b>: La reducción en la producción (Y ↓) disminuye la demanda de dinero, reduciendo la tasa de interés (i ↓).<br>4. <b>Mercado Cambiario</b>: La menor tasa de interés provoca salida de capitales, depreciando el tipo de cambio (e ↑).<br>5. <b>Resultado Final</b>: Nuevo equilibrio con menor producción (Y₁ < Y₀) y un tipo de cambio más depreciado (e₁ > e₀).`;
          }

      } else if (shock_type === 'Monetario') {
          const nuevo_aa_i = AA_I + shock_value;
          y_eq2 = (nuevo_aa_i - DD_I) / (DD_P + AA_P);
          e_eq2 = DD_I + DD_P * y_eq2;
          i_eq2 = (IP_I - e_eq2) / IP_P;

          // Actualiza la fuente de datos de la nueva curva AA y las flechas.
          source_aa_n.data = { x: RANGO_Y, y: RANGO_Y.map(y => nuevo_aa_i - AA_P * y) };
          source_flecha_mc.data = { x_start: [i_eq1], y_start: [e_eq1], x_end: [i_eq2], y_end: [e_eq2] };
          source_flecha_ddaa.data = { x_start: [y_eq1], y_start: [e_eq1], x_end: [y_eq2], y_end: [e_eq2] };

          if (shock_value > 0) {
              titulo = "<h3>Mecanismo de Ajuste: Política Monetaria Expansiva</h3>";
              cuerpo = `1. <b>Inicio</b>: El banco central aumenta la oferta monetaria (Mˢ/P ↑).<br>2. <b>Mercado de Dinero</b>: Exceso de oferta de dinero, lo que reduce la tasa de interés (i ↓).<br>3. <b>Mercado Cambiario</b>: La menor tasa de interés provoca salida de capitales, depreciando el tipo de cambio (e ↑).<br>4. <b>Modelo DD-AA</b>: El aumento de Mˢ desplaza la curva AA hacia arriba.<br>5. <b>Resultado Final</b>: Nuevo equilibrio con mayor producción (Y₁ > Y₀) y un tipo de cambio más depreciado (e₁ > e₀).`;
          } else if (shock_value < 0) {
              titulo = "<h3>Mecanismo de Ajuste: Política Monetaria Contractiva</h3>";
              cuerpo = `1. <b>Inicio</b>: El banco central reduce la oferta monetaria (Mˢ/P ↓).<br>2. <b>Mercado de Dinero</b>: Exceso de demanda de dinero, lo que eleva la tasa de interés (i ↑).<br>3. <b>Mercado Cambiario</b>: La mayor tasa de interés atrae capitales, apreciando el tipo de cambio (e ↓).<br>4. <b>Modelo DD-AA</b>: La reducción de Mˢ desplaza la curva AA hacia abajo.<br>5. <b>Resultado Final</b>: Nuevo equilibrio con menor producción (Y₁ < Y₀) y un tipo de cambio más apreciado (e₁ < e₀).`;
          }

      } else if (shock_type === 'Cambiario') {
          const nuevo_aa_i = AA_I + shock_value;
          const nuevo_ip_i = IP_I + shock_value;
          y_eq2 = (nuevo_aa_i - DD_I) / (DD_P + AA_P);
          e_eq2 = DD_I + DD_P * y_eq2;
          i_eq2 = (nuevo_ip_i - e_eq2) / IP_P;

          // Actualiza la fuente de datos de las nuevas curvas IP y AA, y las flechas.
          source_ip_n.data = { x: RANGO_I, y: RANGO_I.map(i => nuevo_ip_i - IP_P * i) };
          source_aa_n.data = { x: RANGO_Y, y: RANGO_Y.map(y => nuevo_aa_i - AA_P * y) };
          source_flecha_mc.data = { x_start: [i_eq1], y_start: [e_eq1], x_end: [i_eq2], y_end: [e_eq2] };
          source_flecha_ddaa.data = { x_start: [y_eq1], y_start: [e_eq1], x_end: [y_eq2], y_end: [e_eq2] };


          if (shock_value > 0) {
              titulo = "<h3>Mecanismo de Ajuste: Shock de Expectativas Cambiarias (Positivo)</h3>";
              cuerpo = `1. <b>Inicio</b>: Aumenta la expectativa del tipo de cambio futuro (Eᵉ ↑). Se espera una depreciación.<br>2. <b>Mercado Cambiario</b>: Aumenta el retorno esperado de depósitos extranjeros. Salida de capitales deprecia el tipo de cambio (e ↑).<br>3. <b>Curvas de Activos</b>: La curva IP y la curva AA se desplazan hacia arriba.<br>4. <b>Resultado Final</b>: Nuevo equilibrio con mayor producción (Y₁ > Y₀) y un tipo de cambio más depreciado (e₁ > e₀).`;
          } else if (shock_value < 0) {
              titulo = "<h3>Mecanismo de Ajuste: Shock de Expectativas Cambiarias (Negativo)</h3>";
              cuerpo = `1. <b>Inicio</b>: Disminuye la expectativa del tipo de cambio futuro (Eᵉ ↓). Se espera una apreciación.<br>2. <b>Mercado Cambiario</b>: Disminuye el retorno esperado de depósitos extranjeros. Entrada de capitales aprecia el tipo de cambio (e ↓).<br>3. <b>Curvas de Activos</b>: La curva IP y la curva AA se desplazan hacia abajo.<br>4. <b>Resultado Final</b>: Nuevo equilibrio con menor producción (Y₁ < Y₀) y un tipo de cambio más apreciado (e₁ < e₀).`;
          }
      }

      // --- Actualizar datos del nuevo equilibrio si hay un shock ---
      if (shock_type !== 'Ninguno') {
          // Llena los datos del punto de equilibrio final.
          source_eq2.data = { y: [y_eq2], e: [e_eq2], i: [i_eq2] };
          // Llena los datos de las líneas de proyección para el nuevo equilibrio.
          source_eq2_lines_mc.data = { x0: [0, i_eq2], y0: [e_eq2, 0], x1: [i_eq2, i_eq2], y1: [e_eq2, e_eq2] };
          source_eq2_lines_ddaa.data = { x0: [0, y_eq2], y0: [e_eq2, 0], x1: [y_eq2, y_eq2], y1: [e_eq2, e_eq2] };
      }

      // --- Actualizar el texto de explicación y notificar cambios a Bokeh ---
      explanation_div.text = titulo + "<p>" + cuerpo + "</p>";

      // El método .change.emit() notifica a Bokeh que los datos han cambiado y que debe volver a dibujar.
      source_ip_n.change.emit();
      source_dd_n.change.emit();
      source_aa_n.change.emit();
      source_eq2.change.emit();
      source_flecha_mc.change.emit();
      source_flecha_ddaa.change.emit();
      source_eq2_lines_mc.change.emit();
      source_eq2_lines_ddaa.change.emit();
  """
)

# --- Vincular el callback a los widgets ---
# Se le dice a Bokeh que ejecute el `callback` de JavaScript cuando la propiedad `value` de los widgets cambie.
shock_select.js_on_change('value', callback)
shock_slider.js_on_change('value', callback)


# =============================================================================
# --- PASO 7: ORGANIZACIÓN DEL LAYOUT Y GENERACIÓN DEL ARCHIVO FINAL ---
# =============================================================================

# --- Organizar el Layout del Dashboard ---
# Se define la estructura visual del dashboard, apilando los controles, los gráficos y el texto.
controles = column(shock_select, shock_slider)
graficos = row(p_mc, p_ddaa)
dashboard = layout([
    [controles],
    [graficos],
    [explanation_div]
])

# --- Mostrar el Dashboard ---
# Se muestra el layout completo directamente en la salida de la celda de Colab.
show(dashboard)

# Mensaje de confirmación en la consola (este mensaje aparecerá después de que se muestre el dashboard).
print("¡Dashboard generado con éxito!")

¡Dashboard generado con éxito!
